In [1]:
import pandas as pd


In [2]:
DATA_FP = "../cnn_dailymail/3.0.0/train-00000-of-00003.parquet"
NUM_ARTICLES = 100

# We use Milvus vector database for knowledge base
MILVUS_URL = "http://localhost:19530"
MILVUS_HOST = MILVUS_URL.split("//")[-1].split(":")[0]
MILVUS_PORT = MILVUS_URL.split(":")[-1]
MILVUS_COLLECTION_NAME = "articles_collection"
VECTOR_DIM = 384  # Updated to match the new embedding model
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# vLLM OpenAI Compatible Server for LLM
VLLM_HOST = "http://localhost:8000/v1"
LLM_NAME = "gpt2"
MAX_TOKENS = 512  # Updated to match the new LLM
TOP_P = 0.95
TEMPERATURE = 0.01
PRESENCE_PENALTY = 1.03


In [ ]:
# Read the input data
df = pd.read_parquet(DATA_FP)
df = df[["id", "article"]]
# Investigate an article
print(df.iloc[0]["article"])
# Get all articles in a list format
articles = df.iloc[:NUM_ARTICLES].to_dict("records")

# PDF processing

In [1]:
# We use Milvus vector database for knowledge base
MILVUS_URL = "http://localhost:19530"
MILVUS_HOST = MILVUS_URL.split("//")[-1].split(":")[0]
MILVUS_PORT = MILVUS_URL.split(":")[-1]
MILVUS_COLLECTION_NAME = "articles_collection"
VECTOR_DIM = 384  # Updated to match the new embedding model
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# vLLM OpenAI Compatible Server for LLM
VLLM_HOST = "http://localhost:8000/v1"
LLM_NAME = "gpt2"
MAX_TOKENS = 512  # Updated to match the new LLM
TOP_P = 0.95
TEMPERATURE = 0.01
PRESENCE_PENALTY = 1.03

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("/home/khoa/MLE-k3/Supplementary/llm-source-code/camnangsv.pdf")
data = loader.load()

In [3]:
import pymupdf4llm


ModuleNotFoundError: No module named 'pymupdf4llm'

# Ingest data to Milvus

In [4]:
# Ingest data into Milvus (or any vector database of your choice)
from pymilvus import (
    MilvusClient,
    DataType,
)
from sentence_transformers import SentenceTransformer

# Connect to Milvus
milvus_client = MilvusClient(uri=MILVUS_HOST)

# Create a collection to store vectors and metadata,
# think of it as a table in a traditional SQL database.
schema = milvus_client.create_schema(auto_id=False, enable_dynamic_field=False)
schema.add_field(
    schema=schema,
    field_name="id",
    datatype=DataType.VARCHAR,
    is_primary=True,
    max_length=10000,
)
# This is the vector embedding
schema.add_field(
    schema=schema, field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=VECTOR_DIM
)
# This is the corresponding raw text for the vector
schema.add_field(
    schema=schema, field_name="text", datatype=DataType.VARCHAR, max_length=10000
)

# Drop the old collection and create a new one
if milvus_client.has_collection(MILVUS_COLLECTION_NAME):
    milvus_client.drop_collection(MILVUS_COLLECTION_NAME)

milvus_client.create_collection(collection_name=MILVUS_COLLECTION_NAME, schema=schema)

# Define an embedding model to create vectors from texts,
# you can replace by other models
transformer = SentenceTransformer(EMBEDDING_MODEL_NAME)

# Loop over articles and create embedding vectors
data = []
for article in articles:
    data.append(
        {
            "id": article["id"],
            "vector": transformer.encode(article["article"]),
            "text": article["article"],
        }
    )
res = milvus_client.insert(collection_name=MILVUS_COLLECTION_NAME, data=data)

/home/khoa/anaconda3/envs/chatbot-rag/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Query data via Langchain

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms.vllm import VLLMOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores.milvus import Milvus

# We access vLLM via OpenAI compatible server
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=VLLM_HOST,
    model_name=LLM_NAME,
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    streaming=True,
    verbose=False,
)

# We our embedding model reside on cpu
model_kwargs = {"trust_remote_code": True, "device": "cpu"}

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME, model_kwargs=model_kwargs, show_progress=False
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
    collection_name=MILVUS_COLLECTION_NAME,
    text_field="text",
    drop_old=False,
)
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

/tmp/ipykernel_11628/1449126244.py:24: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 0.3.0. An updated version of the class exists in the langchain-milvus package and should be used instead. To use it run `pip install -U langchain-milvus` and import as `from langchain_milvus import MilvusVectorStore`.
  store = Milvus(


In [8]:
# Test our chain (retriever and LLM) with a question
question = "Tell me some news about Harry Potter?"
result = qa_chain.invoke({"query": question})
print(result)

{'query': 'Tell me some news about Harry Potter?', 'result': '\n\nHarry Potter is a fictional character from the Harry Potter series. He is the son of a wizard and a witch. He is the only member of the family to have a surname.\n\nHarry Potter is a fictional character from the Harry Potter series. He is the son of a wizard and a witch. He is the only member of the family to have a surname. Harry Potter is a fictional character from the Harry Potter series. He is the son of a wizard and a witch. He is the only member of the family to have a surname. Harry Potter is a fictional character from the Harry Potter series. He is the son of a wizard and a witch. He is the only member of the family to have a surname. Harry Potter is a fictional character from the Harry Potter series. He is the son of a wizard and a witch. He is the only member of the family to have a surname. Harry Potter is a fictional character from the Harry Potter series. He is the son of a wizard and a witch. He is the only